In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
import math
import time

import numpy as np
import tensorflow as tf

import model_cifar10 as cifar10

In [2]:
FLAGS = tf.app.flags.FLAGS

tf.app.flags.DEFINE_string('eval_dir', '/tmp/cifar10_eval',
                           """Directory where to write event logs.""")
tf.app.flags.DEFINE_string('eval_data', 'test',
                           """Either 'test' or 'train_eval'.""")
tf.app.flags.DEFINE_string('checkpoint_dir', '/tmp/cifar10_train',
                           """Directory where to read model checkpoints.""")
tf.app.flags.DEFINE_integer('eval_interval_secs', 60 * 5,
                            """How often to run the eval.""")
tf.app.flags.DEFINE_integer('num_examples', 10,
                            """Number of examples to run.""")
tf.app.flags.DEFINE_boolean('run_once', False,
                         """Whether to run eval only once.""")

In [3]:
def eval_once(saver, summary_writer, top_k_op, summary_op):
  """Run Eval once.
  Args:
    saver: Saver.
    summary_writer: Summary writer.
    top_k_op: Top K op.
    summary_op: Summary op.
  """
  with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state(FLAGS.checkpoint_dir)
    if ckpt and ckpt.model_checkpoint_path:
      # Restores from checkpoint
      saver.restore(sess, ckpt.model_checkpoint_path)
      # Assuming model_checkpoint_path looks something like:
      #   /my-favorite-path/cifar10_train/model.ckpt-0,
      # extract global_step from it.
      global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
    else:
      print('No checkpoint file found')
      return

    # Start the queue runners.
    coord = tf.train.Coordinator()
    try:
      threads = []
      for qr in tf.get_collection(tf.GraphKeys.QUEUE_RUNNERS):
        threads.extend(qr.create_threads(sess, coord=coord, daemon=True,
                                         start=True))

      num_iter = int(math.ceil(FLAGS.num_examples / FLAGS.batch_size))
      true_count = 0  # Counts the number of correct predictions.
      total_sample_count = num_iter * FLAGS.batch_size
      step = 0
      while step < num_iter and not coord.should_stop():
        predictions = sess.run([top_k_op])
        true_count += np.sum(predictions)
        step += 1

      # Compute precision @ 1.
      precision = true_count / total_sample_count
      print('%s: precision @ 1 = %.3f' % (datetime.now(), precision))

      summary = tf.Summary()
      summary.ParseFromString(sess.run(summary_op))
      summary.value.add(tag='Precision @ 1', simple_value=precision)
      summary_writer.add_summary(summary, global_step)
    except Exception as e:  # pylint: disable=broad-except
      coord.request_stop(e)

    coord.request_stop()
    coord.join(threads, stop_grace_period_secs=10)

In [4]:
def evaluate():
  """Eval CIFAR-10 for a number of steps."""
  with tf.Graph().as_default() as g:
    # Get images and labels for CIFAR-10.
    eval_data = FLAGS.eval_data == 'test'
    images, labels = cifar10.inputs(eval_data=eval_data)

    # Build a Graph that computes the logits predictions from the
    # inference model.
    logits = cifar10.inference(images)

    # Calculate predictions.
    top_k_op = tf.nn.in_top_k(logits, labels, 1)

    # Restore the moving average version of the learned variables for eval.
    variable_averages = tf.train.ExponentialMovingAverage(
        cifar10.MOVING_AVERAGE_DECAY)
    variables_to_restore = variable_averages.variables_to_restore()
    saver = tf.train.Saver(variables_to_restore)

    # Build the summary operation based on the TF collection of Summaries.
    summary_op = tf.summary.merge_all()

    summary_writer = tf.summary.FileWriter(FLAGS.eval_dir, g)

    while True:
      eval_once(saver, summary_writer, top_k_op, summary_op)
      if FLAGS.run_once:
        break
      time.sleep(FLAGS.eval_interval_secs)

In [ ]:
def main(argv=None):  # pylint: disable=unused-argument
  cifar10.maybe_download_and_extract()
  if tf.gfile.Exists(FLAGS.eval_dir):
    tf.gfile.DeleteRecursively(FLAGS.eval_dir)
  tf.gfile.MakeDirs(FLAGS.eval_dir)
  evaluate()
    
if __name__ == '__main__':
  tf.app.run()

In [ ]:
'''
2017-04-02 08:51:36.147992: precision @ 1 = 0.797
2017-04-02 08:56:37.105828: precision @ 1 = 0.805
2017-04-02 09:01:38.081487: precision @ 1 = 0.805
2017-04-02 09:06:38.968899: precision @ 1 = 0.812
2017-04-02 09:11:39.916435: precision @ 1 = 0.812
2017-04-02 09:16:40.838525: precision @ 1 = 0.805
2017-04-02 09:21:41.670195: precision @ 1 = 0.805
2017-04-02 09:26:42.526939: precision @ 1 = 0.805
2017-04-02 09:31:43.496605: precision @ 1 = 0.805
2017-04-02 09:36:44.375324: precision @ 1 = 0.789
2017-04-02 09:41:45.352660: precision @ 1 = 0.797
2017-04-02 09:46:46.351343: precision @ 1 = 0.805
2017-04-02 09:51:47.274158: precision @ 1 = 0.805
2017-04-02 09:56:48.044474: precision @ 1 = 0.812
2017-04-02 10:01:48.851925: precision @ 1 = 0.789
2017-04-02 10:06:49.723382: precision @ 1 = 0.805
2017-04-02 10:11:50.792973: precision @ 1 = 0.812
2017-04-02 10:16:51.682859: precision @ 1 = 0.805
2017-04-02 10:21:52.522326: precision @ 1 = 0.789
2017-04-02 10:26:53.490007: precision @ 1 = 0.805
2017-04-02 10:31:54.351439: precision @ 1 = 0.789
2017-04-02 10:36:55.180113: precision @ 1 = 0.797
2017-04-02 10:41:56.082135: precision @ 1 = 0.805
2017-04-02 10:46:56.936563: precision @ 1 = 0.789
2017-04-02 10:51:57.796806: precision @ 1 = 0.797
2017-04-02 10:56:58.651325: precision @ 1 = 0.805
2017-04-02 11:01:59.491103: precision @ 1 = 0.789
2017-04-02 11:07:00.281066: precision @ 1 = 0.820
2017-04-02 11:12:01.370458: precision @ 1 = 0.805
2017-04-02 11:17:02.323520: precision @ 1 = 0.781
2017-04-02 11:22:03.194257: precision @ 1 = 0.797
2017-04-02 11:27:04.037562: precision @ 1 = 0.805
2017-04-02 11:32:05.037362: precision @ 1 = 0.812
2017-04-02 11:37:05.921188: precision @ 1 = 0.805
2017-04-02 11:42:06.766411: precision @ 1 = 0.805
2017-04-02 11:47:07.641482: precision @ 1 = 0.805
2017-04-02 11:52:08.641175: precision @ 1 = 0.797
2017-04-02 11:57:09.557263: precision @ 1 = 0.805
2017-04-02 12:02:10.450332: precision @ 1 = 0.789
2017-04-02 12:07:11.394514: precision @ 1 = 0.812
2017-04-02 12:12:12.253868: precision @ 1 = 0.805
2017-04-02 12:17:13.207566: precision @ 1 = 0.805
2017-04-02 12:22:14.040418: precision @ 1 = 0.805
2017-04-02 12:27:15.020390: precision @ 1 = 0.820
2017-04-02 12:32:15.947433: precision @ 1 = 0.812
'''